# Install Rapids First

- Note use only T4 or P100 Or P4 GPU which is compatible for RAPIDS.

In [1]:
!nvidia-smi

Sat Mar 21 09:51:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
! cat /proc/cpuinfo

In [2]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 44 (delta 9), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (44/44), done.
PLEASE READ
********************************************************************************************************
Colab Notebooks Migration Notice:
 
We have changed the location of the Colab script to our new CSP utilties repo.
We are also dropping support for versions below 0.11, as we are on 0.13 nightlies
 
Not all Colab notebooks are updated (like personal Colabs) and/or are using outdated versions of RAPIDS,
so we HIGHLY encourage users to update their scripts and Colab notebooks both with the new link AND with 
to the new API.  Otherwise, your code MAY BREAK
 
This notice will disappear/be updated on our next release.
 
Please enter in the box your desired RAPIDS version (ex: '0.11' or '0.12', between 0.11 to

# XGBoost 1.0 Features and Intro

New features in XGBoost 1.0

- XGBoost supports distributed Computing on Kubernetes
- We can use Kubernetes now.
- All cloud such as AWS, can use xGBoost
- Native Dask interface on XGBoost.
- Supports Cudf on RAPIDS.
- Already it supports numpy, pandas, libsvm format.
- Now it supports CUDA Dataframe.
- XGBoost works on GPU already. We can use Cuda Dataframe with end to end on GPU.
- Lot of GPU additions.
- Learning to rank is GPU accelerated.

Internally uses lambda mart ranking algorithm. Uses pairwise ranking approach by sampling many pairs.

- Scaling for multi-core CPUs is enhanced.


## Loading Data

In [0]:
import cudf

In [0]:
! pip install xgboost

In [7]:
import xgboost as xgb
print(xgb.__version__)

1.0.2


In [0]:
import sys,tempfile, urllib, os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [0]:
from sklearn.datasets import fetch_openml
covtyp = fetch_openml(name='covertype', version=4)
# Predicting the forest cover type using 53 variables.
# Predict one categorical class. 

In [10]:
covtyp.data.shape

(581012, 54)

In [13]:
np.unique(covtyp.target)

array(['1', '2', '3', '4', '5', '6', '7'], dtype=object)

In [0]:
cov_df = pd.DataFrame(data=np.c_[covtyp['data'], covtyp['target']], columns=covtyp['feature_names'] + ['target'])

In [12]:
cov_df.memory_usage().sum()

255645408

In [14]:
cov_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,target
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


In [15]:
cov_df.target.value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: target, dtype: int64

In [0]:
cov_df.dtypes

- Convert all objects into float32 format.
- Keep the categorical target as int32

In [0]:
for cols in cov_df.columns:
    cov_df[cols] = cov_df[cols].astype(np.float32)

In [0]:
cov_df.dtypes

In [0]:
cov_df['target'] = cov_df['target'].astype(np.int32)

In [0]:
cov_df.dtypes

- Keep target variable from 0 - 7 instead of 1 - 8

In [0]:
cov_df['target'] = cov_df['target'] - 1

In [0]:
cov_df_x = cov_df.drop(['target'], axis=1)

In [0]:
cov_df_y = pd.DataFrame(cov_df['target'])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(cov_df_x, cov_df_y, train_size=0.75, stratify=cov_df_y, random_state=31)

# Data GPU + XGBoost GPU

In [0]:
X_train_gdf = cudf.DataFrame.from_pandas(X_train)
X_test_gdf = cudf.DataFrame.from_pandas(X_test)
y_train_gdf = cudf.DataFrame.from_pandas(y_train)
y_test_gdf = cudf.DataFrame.from_pandas(y_test)

In [0]:
!nvidia-smi

In [0]:
# New in 1.0 we can directly use the cuda dataframe
dtrain = xgb.DMatrix(X_train_gdf, label=y_train_gdf)
dtest = xgb.DMatrix(X_test_gdf, label=y_test_gdf)

In [37]:
!nvidia-smi

Sat Mar 21 10:35:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    32W / 250W |   1777MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import time

In [0]:
num_rounds = 200
max_depth = 10
params = {
    'colsample_bylevel' : 1,
    'colsample_bytree' : 1,
    'learning_rate' : 0.1,
    'random_state': 31,
    'objective': 'multi:softmax',
    'num_class' : 7
}

In [0]:
params['tree_method'] = 'gpu_hist'
params['grow_policy'] = 'depthwise'
params['max_depth'] = max_depth
params['max_leaves'] = 0
params['verbosity'] = 0
params['gpu_id'] = 0
params['updater'] = 'grow_gpu_hist'
params['predictor'] = 'gpu_predictor'

In [45]:
gpu_result = {}
start_time = time.time()
# Training it
xgb.train(params, dtrain, num_rounds, evals = {(dtest, 'test')}, evals_result=gpu_result, verbose_eval=20)
end_time = time.time()  
print("GPU Time taken to train = %s"%(end_time - start_time))

[0]	test-merror:0.17838
[20]	test-merror:0.13971
[40]	test-merror:0.12770
[60]	test-merror:0.11508
[80]	test-merror:0.10647
[100]	test-merror:0.10054
[120]	test-merror:0.09466
[140]	test-merror:0.08770
[160]	test-merror:0.08068
[180]	test-merror:0.07638
[199]	test-merror:0.07186
Time taken to train = 41.82584619522095


# Data RAM + XGBoost GPU

- NOTE: Not much time taken to move data from RAM to GPU 

In [0]:
# Training on GPU but with data on RAM

dtrain_cpu = xgb.DMatrix(X_train, label=y_train)
dtest_cpu = xgb.DMatrix(X_test, label=y_test)

In [0]:
num_rounds = 200
max_depth = 10
params = {
    'colsample_bylevel' : 1,
    'colsample_bytree' : 1,
    'learning_rate' : 0.1,
    'random_state': 31,
    'objective': 'multi:softmax',
    'num_class' : 7
}

In [0]:
params['tree_method'] = 'gpu_hist'
params['grow_policy'] = 'depthwise'
params['max_depth'] = max_depth
params['max_leaves'] = 0
params['verbosity'] = 0
params['gpu_id'] = 0
params['updater'] = 'grow_gpu_hist'
params['predictor'] = 'gpu_predictor'

In [50]:
gpu_result = {}
start_time = time.time()
# Training it
xgb.train(params, dtrain, num_rounds, evals = {(dtest, 'test')}, evals_result=gpu_result, verbose_eval=20)
end_time = time.time()  
print("RAM Time taken to train = %s" %(end_time - start_time))

[0]	test-merror:0.17838
[20]	test-merror:0.13971
[40]	test-merror:0.12770
[60]	test-merror:0.11508
[80]	test-merror:0.10647
[100]	test-merror:0.10054
[120]	test-merror:0.09466
[140]	test-merror:0.08770
[160]	test-merror:0.08068
[180]	test-merror:0.07638
[199]	test-merror:0.07186
RAM Time taken to train = 41.88491249084473


In [51]:
! nvidia-smi

Sat Mar 21 10:46:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    40W / 250W |   2433MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Data RAM + XGBoost CPU

In [0]:
dtrain_cpu = xgb.DMatrix(X_train, label=y_train)
dtest_cpu = xgb.DMatrix(X_test, label=y_test)

In [0]:
params = {}

In [0]:
num_rounds = 200
max_depth = 10

params = {
    'colsample_bylevel' : 1,
    'colsample_bytree' : 1,
    'learning_rate' : 0.1,
    'random_state': 31,
    'objective': 'multi:softmax',
    'num_class' : 7
}

In [0]:
params['tree_method'] = 'hist'
params['grow_policy'] = 'depthwise'
params['max_depth'] = max_depth
params['max_leaves'] = 0
params['verbosity'] = 0
# params['gpu_id'] = 0
# params['updater'] = 'hist'
params['predictor'] = 'cpu_predictor'

In [67]:
cpu_result = {}
start_time = time.time()
# Training it
xgb.train(params, dtrain, num_rounds, evals = {(dtest, 'test')}, evals_result=cpu_result, verbose_eval=20)
end_time = time.time()  
print("RAM Time taken to train = %s" %(end_time - start_time))

[0]	test-merror:0.17823
[20]	test-merror:0.13898
[40]	test-merror:0.12763
[60]	test-merror:0.11415
[80]	test-merror:0.10697
[100]	test-merror:0.10146
[120]	test-merror:0.09487
[140]	test-merror:0.08651
[160]	test-merror:0.08054
[180]	test-merror:0.07401
[199]	test-merror:0.06994
RAM Time taken to train = 314.41554594039917


# Which to use and When 

- If data is not big use pandas directly
- Pandas works on single node so has limitation with bigger datasets.
- For bigger datasets like 3GB if we have GPU available use RAPIDS and directly make a CuDf it will be faster and efficient.
- Pandas will give performance issues with huge data.
- If data is not fitting into single GPU then use distributed GPU's or use distributed frameworks like Dask.
- For CPU only training one can use Intel's DAAL framework.